In [6]:
!pip install -q ipython-autotime
%load_ext autotime


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.91 s (started: 2024-04-12 10:52:24 +00:00)


In [7]:
# Install Pytorch & other libraries
!pip install "torch==2.1.2" tensorboard pillow wandb

# Install Hugging Face libraries
!pip install -qqq git+https://github.com/huggingface/transformers
!pip install -qqq git+https://github.com/huggingface/datasets
!pip install -qqq git+https://github.com/huggingface/peft
!pip install -qqq git+https://github.com/huggingface/accelerate
!pip install -qqq --upgrade bitsandbytes
!pip install -qqq git+https://github.com/huggingface/trl
!pip install -qqq --upgrade safetensors
!pip install -qqq -U flash-attn
!pip install -qqq evaluate
!pip install -qqq gputil psutil
!pip install -qqq wandb



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
time: 1min 35s (started: 2024-04-12 10:52:27 +00:00)


In [8]:
import torch; assert torch.cuda.get_device_capability()[0] >= 8, 'Hardware not supported for Flash Attention'
# install flash-attn
!pip install ninja packaging
!MAX_JOBS=4 pip install flash-attn --no-build-isolation



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
time: 5.52 s (started: 2024-04-12 10:54:03 +00:00)


In [9]:
import wandb

time: 458 µs (started: 2024-04-12 10:54:09 +00:00)


In [10]:
wandb.init(project="doplhin_mistral_dpo_ultrafeedback", entity="drishtisharma96505", group='orpo')

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


time: 3min 4s (started: 2024-04-12 10:54:09 +00:00)


In [11]:
from transformers import AutoTokenizer
from datasets import load_dataset


time: 957 ms (started: 2024-04-12 10:57:16 +00:00)


In [12]:
# Load Tokenizer from the hub
model_id = "cognitivecomputations/dolphin-2.1-mistral-7b" # replace with your model id
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/100 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


time: 2.26 s (started: 2024-04-12 10:57:23 +00:00)


In [13]:
# Load dataset from the hub
dataset = load_dataset("argilla/ultrafeedback-binarized-preferences-cleaned", split="train")
dataset = dataset.shuffle().select(range(13750))

Generating train split:   0%|          | 0/60917 [00:00<?, ? examples/s]

time: 3.93 s (started: 2024-04-12 10:57:32 +00:00)


In [14]:
print(dataset)

Dataset({
    features: ['source', 'prompt', 'chosen', 'chosen-rating', 'chosen-model', 'rejected', 'rejected-rating', 'rejected-model'],
    num_rows: 13750
})
time: 1.09 ms (started: 2024-04-12 10:57:36 +00:00)


In [16]:
dataset[0]

{'source': 'flan_v2_p3',
 'prompt': "Question: Read the passage below and choose the right answer to the following question (choices are America or China ):  Countries with more industry produce more hazardous wastes than those with little industry.   James loved to travel the world. In his travels, he noticed that America contained a lot of industry while China contained not much industry at all. Which country produced more hazardous wastes America or China?\nAnswer: America\n\nQuestion: Read the passage below and choose the right answer to the following question (choices are bigger or smaller ):  If you are standing on the shore and a ship is going out to sea, the ship gets smaller as it moves further away from you.   Dan is watching the ship leave the dock. While the ship is near him, it is massive. However, as the ship floats away, it begins to look bigger or smaller?\nAnswer: smaller\n\nQuestion: Read the passage below and choose the right answer to the following question (choices

time: 9.1 ms (started: 2024-04-12 10:57:41 +00:00)


In [17]:
def rec_extract_assistant_messages(messages, index=-1):
  """Recursively extract the last assistant messages from the end of the conversation."""
  if messages[index]["role"] == "assistant":
    return [messages[index]]
  else:
    return rec_extract_assistant_messages(messages, index-1)

# System message used if there is no system message at the beginning of the conversation
# Can be repelaced and modified as needed
DEFAULT_SYSTEM_MESSAGE = "You are Dolphin, a helpful AI assistant."

def create_triplets(example, tokenizer, default_system_message=DEFAULT_SYSTEM_MESSAGE):
  """Create the triplets (prompt, chosen, rejected)"""
  # Extract the N-1 turns to form the prompt
  # Prepend a system message if the first message is not a system message
  prompt_messages = example["chosen"][:-1]
  if example["chosen"][0]["role"] != "system":
      prompt_messages.insert(0, {"role": "system", "content": default_system_message})
  # Now we extract the final assistant turn to define chosen/rejected responses
  chosen_messages = rec_extract_assistant_messages(example["chosen"])
  rejected_messages = rec_extract_assistant_messages(example["rejected"])

  # apply template to the messages and return the triplets
  return {
    "prompt": tokenizer.apply_chat_template(prompt_messages, tokenize=False),
    "chosen": tokenizer.apply_chat_template(chosen_messages, tokenize=False),
    "rejected": tokenizer.apply_chat_template(rejected_messages, tokenize=False)
  }


time: 2.87 ms (started: 2024-04-12 10:57:48 +00:00)


In [18]:
dataset = dataset.map(create_triplets, remove_columns=dataset.features, fn_kwargs={"tokenizer": tokenizer})
# split dataset into 11,000 training samples and 2,750 test samples
dataset = dataset.train_test_split(test_size=2750/13750)

# print sample cut of
print(dataset["train"][0]["prompt"][:50])
print(dataset["train"][0]["chosen"][:50])
print(dataset["train"][0]["rejected"][:50])

# save datasets to disk
dataset["train"].to_json("train_dataset.json", orient="records")
dataset["test"].to_json("test_dataset.json", orient="records")

Map:   0%|          | 0/13750 [00:00<?, ? examples/s]

<|im_start|>system
You are Dolphin, a helpful AI a
<|im_start|>assistant
Metamorphosis is a process w
<|im_start|>assistant
Metamorphosis is the process


Creating json from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

9546409

time: 4.43 s (started: 2024-04-12 10:57:52 +00:00)


In [19]:
from datasets import load_dataset

# Load jsonl data from disk
train_dataset = load_dataset("json", data_files="train_dataset.json", split="train")
eval_dataset = load_dataset("json", data_files="test_dataset.json", split="train")


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

time: 524 ms (started: 2024-04-12 10:57:58 +00:00)


In [20]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Hugging Face model id
model_id = "cognitivecomputations/dolphin-2.1-mistral-7b" # replace with your model id

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    use_cache=False,
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


time: 5min 29s (started: 2024-04-12 10:58:30 +00:00)


In [21]:
print(tokenizer)

LlamaTokenizerFast(name_or_path='cognitivecomputations/dolphin-2.1-mistral-7b', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '<|im_end|>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32000: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32001: AddedToken("<|im_start|>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=True),
}
time: 2.86 ms (started: 2024-04-12 11:03:59 +00:00)


In [22]:
print(tokenizer.pad_token,tokenizer.eos_token)

None <|im_end|>
time: 2.49 ms (started: 2024-04-12 11:03:59 +00:00)


In [23]:
tokenizer.pad_token = tokenizer.eos_token

time: 1.04 ms (started: 2024-04-12 11:03:59 +00:00)


In [24]:
tokenizer.truncation_side

'right'

time: 3.38 ms (started: 2024-04-12 11:03:59 +00:00)


In [25]:
tokenizer.truncation_side = 'left' # to prevent cutting off last generation

time: 761 µs (started: 2024-04-12 11:03:59 +00:00)


In [26]:
prompt_length = 1024
max_seq_length = 1512

time: 2.07 ms (started: 2024-04-12 11:03:59 +00:00)


In [27]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32002, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralFlashAttention2(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
   

time: 5.57 ms (started: 2024-04-12 11:03:59 +00:00)


In [28]:
model.config

MistralConfig {
  "_name_or_path": "cognitivecomputations/dolphin-2.1-mistral-7b",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method": "bitsandbytes"
  },
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "slidi

time: 3.67 ms (started: 2024-04-12 11:03:59 +00:00)


In [29]:
from peft import LoraConfig

# LoRA config based on QLoRA paper & Sebastian Raschka experiment
peft_config = LoraConfig(
        lora_alpha=64,
        lora_dropout=0.0,
        r=64,
        bias="none",
        target_modules="all-linear",
        task_type="CAUSAL_LM",
)


time: 39.7 ms (started: 2024-04-12 11:03:59 +00:00)


In [30]:
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 11000
    })
    test: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 2750
    })
})

time: 3.89 ms (started: 2024-04-12 11:03:59 +00:00)


In [31]:
from transformers import TrainingArguments
from trl import ORPOTrainer, ORPOConfig


orpo_config = ORPOConfig(
    output_dir=f"doplhin-2.1-mistral-7b-orpo-ultrafeedback-binarized-preferences",               # directory to save and repository id
    num_train_epochs=1,                     # number of training epochs
    per_device_train_batch_size=4,         # batch size per device during training
    per_device_eval_batch_size=4,           # batch size for evaluation
    gradient_accumulation_steps=1,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch_fused",              # use fused adamw optimizer
    learning_rate=5e-5,                     # 10x higher LR than QLoRA paper
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.1,                       # warmup ratio based on QLoRA paper
    lr_scheduler_type="cosine",             # use cosine learning rate scheduler
    logging_steps=25,                       # log every 25 steps
    save_steps=500,                         # when to save checkpoint
    save_total_limit=2,                     # limit the total amount of checkpoints
    evaluation_strategy="steps",            # evaluate every 1000 steps
    eval_steps=700,                         # when to evaluate
    beta = 0.1,
    bf16=True,                              # use bfloat16 precision
    tf32=True,                              # use tf32 precision
    push_to_hub=True,                      # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
)




time: 105 ms (started: 2024-04-12 11:12:00 +00:00)


In [34]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): Traceback (most recent call last):
  File "/usr/local/bin/huggingface-cli", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/huggingface_cli.py", line 49, in main
    service.run()
  File "/usr/local/lib/python3

In [35]:
from huggingface_hub import notebook_login
notebook_login()

time: 23.4 ms (started: 2024-04-12 11:15:07 +00:00)


In [38]:
trainer = ORPOTrainer(
    model,
    peft_config=peft_config,
    args=orpo_config,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)


/usr/local/lib/python3.10/dist-packages/trl/trainer/orpo_trainer.py:209: UserWarning: `max_length` is not set in the ORPOConfig's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/orpo_trainer.py:218: UserWarning: `max_prompt_length` is not set in the ORPOConfig's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/orpo_trainer.py:247: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/11000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2750 [00:00<?, ? examples/s]

time: 1min 32s (started: 2024-04-12 11:15:45 +00:00)


In [39]:
# Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100 80GB PCIe. Max memory = 79.15 GB.
5.762 GB of memory reserved.
time: 1.91 ms (started: 2024-04-12 11:17:17 +00:00)


In [40]:
import time

time: 2.09 ms (started: 2024-04-12 11:17:17 +00:00)


In [41]:
start_time = time.time()  # Start timer

trainer_stats = trainer.train()

end_time = time.time()  # End timer
training_duration = (end_time - start_time)/60

print(f"Training Time: {training_duration} minutes")

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Runtime,Samples Per Second,Steps Per Second,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen,Nll Loss,Log Odds Ratio,Log Odds Chosen
700,0.910100,0.884483,442.259100,6.218000,1.556000,-0.086941,-0.110594,0.642805,0.023653,-1.105936,-0.869406,-2.663075,-2.743107,0.822386,-0.622538,0.363116
1400,0.855400,0.860901,443.136000,6.206000,1.553000,-0.087662,-0.123316,0.655523,0.035655,-1.233165,-0.876615,-2.616893,-2.699567,0.800671,-0.604029,0.504829
2100,0.901100,0.850565,442.686400,6.212000,1.554000,-0.085246,-0.116645,0.645712,0.031399,-1.166451,-0.852462,-2.651705,-2.724993,0.789643,-0.611014,0.458075


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

Training Time: 111.79738516807556 minutes
time: 1h 51min 47s (started: 2024-04-12 11:17:17 +00:00)


In [ ]:
trainer.push_to_hub()

events.out.tfevents.1712920638.eace2362923f.1330.0:   0%|          | 0.00/103k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]